# Tunix Zero-Cost Submission


## Your overall training and evaluation strategy

**Strategy: Format-Align-Reinforce (Zero-Cost)**
Our goal is to fit a full reasoning distillation pipeline into a single 9-hour TPU session using only public data.
1.  **Format (SFT)**: We leverage the strong pre-trained instructions of `Gemma-2-2b-it`.
2.  **Reinforce (GRPO)**: We use Tunix GRPO on `GSM8K` (Math) and `MBPP` (Code) to optimize for correctness using a memory-efficient group relative policy.

**Evaluation**:
We use a custom "Judge" script to verify the presence of reasoning traces and correct answers locally. In this notebook, we perform a final sanity check generation.

## 🗺️ Workflow Diagram
```mermaid
graph LR
    A[Public Data] --> B(SFT Phase)
    B --> C{GRPO Phase}
    C -->|Math| D[GSM8K]
    C -->|Code| E[MBPP]
    D & E --> F[Final Policy]
    F --> G[Submission]
```



## How your finetuning dataset is created

We employ a **Zero-Cost Public Data Strategy**.
- **Magpie-Reasoning**: Filtered for high-quality reasoning traces, formatted into XML.
- **UltraFeedback**: Used for style alignment (chosen vs rejected).
- **GSM8K & MBPP**: Standard datasets formatted for GRPO (Prompt vs Ground Truth).
All datasets are pre-processed and uploaded as a Kaggle Dataset to save runtime.


## Your Tunix finetuning code

In [ ]:

# Your prompt
PROMPT_TEMPLATE = "<start_of_turn>user\n{question}<end_of_turn>\n<start_of_turn>model\n"

# Training parameters
TEMPERATURE=0.7
TOP_K=50
TOP_P=0.9
MAX_GENERATION_STEPS=768

# Output Tags
REASONING_START = "<reasoning>"
REASONING_END = "</reasoning>"
SOLUTION_START = "<answer>"
SOLUTION_END = "</answer>"

# Inference Params
INF_TEMPERATURE=0
INF_TOP_K=1
INF_TOP_P=None
SEED=42

print("Template variables defined.")


In [ ]:

# --- Setup & Install ---
!pip install -q wandb==0.22.0
!pip install -q kagglehub
!pip install -q ipywidgets
!pip install -q tensorflow
!pip install -q tensorflow_datasets
!pip install -q tensorboardX
!pip install -q transformers
!pip install -q grain
!pip install "google-tunix[prod]==0.1.5"
!pip install git+https://github.com/google/qwix

# Fix Flax Version to 0.12.0 as required
!pip uninstall -q -y flax
!pip install flax==0.12.0

!pip install -q datasets==3.2.0 optax==0.2.4 chex==0.1.88

# --- Imports ---
import functools
import gc
import os
from pprint import pprint
import re
import csv
import shutil
import time

from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
from pathlib import Path
import qwix
import tensorflow_datasets as tfds
import datasets
from tqdm.auto import tqdm

# Tunix Imports
from tunix.generate import sampler as sampler_lib
from tunix.generate import tokenizer_adapter as tokenizer_lib
from tunix.models.gemma import model as gemma_lib
from tunix.models.gemma import params as params_lib
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GRPOLearner
from tunix.rl.rollout import base_rollout
from tunix.sft import metrics_logger
from tunix.sft import peft_trainer

# Transformers (for utility/check)
from transformers import AutoTokenizer

# --- Stability Configs ---
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.95'
jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")

print(f"JAX Devices: {jax.devices()}")

# --- Configuration Constants ---
PRETRAINED_PATH = None 
MODEL_ID = "google/gemma-2-2b-it"
DATASET_PATH = "/kaggle/input/tunix-public-data" 
SFT_OUTPUT_DIR = "sft_checkpoint"
GRPO_OUTPUT_DIR = "grpo_checkpoint"

# Tuning Hyperparams
SFT_STEPS = 400 
GRPO_STEPS = 1500  # Increased from 600 for better convergence
TRAIN_MICRO_BATCH_SIZE = 1 # Keep low for safety


In [ ]:

# --- Model Utilities ---
MESH = [(8, 1), ("fsdp", "tp")]

def get_gemma_ref_model(ckpt_path):
  mesh = jax.make_mesh(*MESH)
  model_config = gemma_lib.ModelConfig.gemma2_2b()
  abs_gemma: nnx.Module = nnx.eval_shape(
      lambda: gemma_lib.Transformer(model_config, rngs=nnx.Rngs(params=0))
  )
  abs_state = nnx.state(abs_gemma)
  abs_state = jax.tree.map(
      lambda a, s: jax.ShapeDtypeStruct(a.shape, jnp.bfloat16, sharding=s),
      abs_state,
      nnx.get_named_sharding(abs_state, mesh),
  )
  checkpointer = ocp.StandardCheckpointer()
  restored_params = checkpointer.restore(ckpt_path, target=abs_state)

  graph_def, _ = nnx.split(abs_gemma)
  gemma = nnx.merge(graph_def, restored_params)
  return gemma, mesh, model_config

def get_lora_model(base_model, mesh):
  # Tunix LoRA Config
  RANK = 64
  ALPHA = 64.0
  lora_provider = qwix.LoraProvider(
      module_path=(
          ".*q_einsum|.*kv_einsum|.*gate_proj|.*down_proj|.*up_proj|"
          ".*attn_vec_einsum"
      ),
      rank=RANK,
      alpha=ALPHA,
  )

  model_input = base_model.get_model_input()
  lora_model = qwix.apply_lora_to_model(
      base_model, lora_provider, rngs=nnx.Rngs(params=0), **model_input
  )

  with mesh:
    state = nnx.state(lora_model)
    pspecs = nnx.get_partition_spec(state)
    sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
    nnx.update(lora_model, sharded_state)

  return lora_model


In [ ]:

# --- Optional: WandB Logging ---
try:
    import wandb
    from kaggle_secrets import UserSecretsClient

    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("WANDB_API_KEY")

    if secret_value:
        wandb.login(key=secret_value)
        wandb.init(project="tunix-zero-cost", name="golden-run-v1", anonymous="allow")
        print("WandB Logging Enabled.")
    else:
        raise ValueError("Empty WANDB_API_KEY")

except Exception as e:
    print(f"WandB not enabled: {e}")
    os.environ["WANDB_MODE"] = "disabled"
    if 'wandb' in locals():
        wandb.init = lambda *args, **kwargs: None
    print("Proceeding without cloud logging (WANDB_MODE='disabled').")


In [ ]:

# --- Logic: Model Prep & GRPO Training ---

# 1. Download/setup Base Model
if "KAGGLE_USERNAME" not in os.environ:
    kagglehub.login()

# Download Gemma 2 (Flax)
model_path = { "gemma2": "google/gemma-2/flax/" }
model_family = "gemma2"
model_version = "gemma2-2b-it" 
kaggle_ckpt_path = kagglehub.model_download(f"{model_path[model_family]}{model_version}")

# Convert checkpoint format for Tunix/NNX
INTERMEDIATE_CKPT_DIR = "/tmp/content/intermediate_ckpt/"
CKPT_DIR = "/tmp/content/ckpts/"
!rm -rf {INTERMEDIATE_CKPT_DIR} {CKPT_DIR}

params = params_lib.load_and_format_params(os.path.join(kaggle_ckpt_path, "gemma2-2b-it"))
gemma = gemma_lib.Transformer.from_params(params, version="2-2b-it")
checkpointer = ocp.StandardCheckpointer()
_, state = nnx.split(gemma)
checkpointer.save(os.path.join(INTERMEDIATE_CKPT_DIR, "state"), state)
checkpointer.wait_until_finished()
del params, gemma, state
gc.collect()

# 2. Load Models
ref_model, mesh, model_config = get_gemma_ref_model(os.path.join(INTERMEDIATE_CKPT_DIR, "state"))
lora_policy = get_lora_model(ref_model, mesh=mesh)

# 3. Setup Tokenizer
tokenizer = tokenizer_lib.Tokenizer(
    tokenizer_path=os.path.join(kaggle_ckpt_path, "tokenizer.model")
)

# 4. Phase 1: SFT (Using Pre-Tuned Model)
# We start with Gemma-2B-IT which has already undergone SFT.
# This allows us to focus our 9h compute budget on Reinforcement Learning (GRPO).

# --- Pre-Training Evaluation (Baseline) ---
print("Running Baseline Evaluation...")
baseline_prompts = [
    "Janet has 3 apples. She buys 2 more. How many does she have now?",
    "Write a python function to add two numbers."
]
try:
    baseline_sampler = sampler_lib.Sampler(
        transformer=ref_model,
        tokenizer=tokenizer,
        cache_config=sampler_lib.CacheConfig(
            cache_size=512,
            num_layers=model_config.num_layers,
            num_kv_heads=model_config.num_kv_heads,
            head_dim=model_config.head_dim,
        ),
    )
    formatted = [TEMPLATE.format(question=p) for p in baseline_prompts]
    baseline_out = baseline_sampler(
        input_strings=formatted,
        max_generation_steps=100,
        temperature=0.7,
        echo=False
    )
    print("--- Baseline Model Outputs (Before Training) ---")
    for p, o in zip(baseline_prompts, baseline_out.text):
        print(f"Q: {p}\nA: {o[:200]}...\n{'-'*40}")
except Exception as e:
    print(f"Baseline eval skipped: {e}")
print("Baseline Done.")

# 5. Phase 2: GRPO
# 5. Phase 2: GRPO
print("Starting GRPO Phase...")
SYSTEM_PROMPT = "You are a deep thinking AI. You are given a problem. Think about the problem and provide your reasoning between <reasoning> and </reasoning> tags. Then, provide the final answer between <answer> and </answer> tags."
TEMPLATE = f"<start_of_turn>user\n{SYSTEM_PROMPT}\n\n{{question}}<end_of_turn>\n<start_of_turn>model"

# --- Reward Functions ---
# 1. Structure Reward: Checks for correct XML tags
# 2. Soft Structure Reward: Partial credit for components
def soft_structure_reward(prompts, completions, **kwargs):
    rewards = []
    for c in completions:
        score = 0.0
        # Check for individual tags
        if "<reasoning>" in c: score += 0.1
        if "</reasoning>" in c: score += 0.1
        if "<answer>" in c: score += 0.1
        if "</answer>" in c: score += 0.1
        
        # Check for content existence
        if re.search(r"<reasoning>.*?</reasoning>", c, re.DOTALL): score += 0.3
        if re.search(r"<answer>.*?</answer>", c, re.DOTALL): score += 0.3
        
        # Max score is 1.0
        rewards.append(min(1.0, score))
    return rewards

# 1. Strict Structure Reward: Checks for correct XML tags (Binary)
def structure_reward(prompts, completions, **kwargs):
    rewards = []
    for c in completions:
        has_reasoning = "<reasoning>" in c and "</reasoning>" in c
        has_answer = "<answer>" in c and "</answer>" in c
        score = 0.5 * has_reasoning + 0.5 * has_answer
        rewards.append(score)
    return rewards

# 2. Math Correctness: Extracts number from answer tag
def math_correctness_reward(prompts, completions, answer, **kwargs):
    rewards = []
    for c, gt in zip(completions, answer):
        try:
            # Extract content between <answer> tags
            match = re.search(r"<answer>(.*?)</answer>", c, re.DOTALL)
            if match:
                extracted = match.group(1).strip()
                # Simple float comparison
                if float(extracted) == float(gt):
                    rewards.append(1.0)
                else:
                    rewards.append(0.0)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
    return rewards

# 3. Code Correctness: Checks if python code is syntactically valid via AST
import ast
def code_correctness_reward(prompts, completions, **kwargs):
    rewards = []
    for c in completions:
        try:
            # Extract code block
            code_match = re.search(r"```python(.*?)```", c, re.DOTALL)
            if code_match:
                code_str = code_match.group(1)
                ast.parse(code_str) # Will raise error if invalid syntax
                rewards.append(1.0)
            else:
                rewards.append(0.0) # No code block found
        except:
            rewards.append(0.0) # Syntax error
    return rewards

# Load Dataset (Robust Path Handling)
if os.path.exists(DATASET_PATH):
    GRID_PATH = DATASET_PATH
else:
    # Fallback/Local path
    GRID_PATH = "data" 
    print(f"Dataset path {DATASET_PATH} not found, checking local {GRID_PATH}...")

try:
    d_math = datasets.load_dataset("json", data_files=f"{GRID_PATH}/grpo_gsm8k_train.jsonl", split="train")
    # Optional coding dataset
    if os.path.exists(f"{GRID_PATH}/grpo_mbpp_train.jsonl"):
         d_code = datasets.load_dataset("json", data_files=f"{GRID_PATH}/grpo_mbpp_train.jsonl", split="train")
         grpo_dataset = datasets.concatenate_datasets([d_math, d_code]).shuffle(seed=42)
    else:
         grpo_dataset = d_math
except Exception as e:
    print(f"Failed to load datasets: {e}")
    # Fallback to dummy data for smoke testing
    grpo_dataset = datasets.Dataset.from_dict({
        "question": ["What is 1+1?", "Write hello world in python."],
        "answer": ["2", "print('hello world')"]
    })

# Optimizer
# Optimizer with Schedule & Clipping
schedule = optax.warmup_cosine_decay_schedule(
    init_value=0.0,
    peak_value=5e-6,
    warmup_steps=100,
    decay_steps=GRPO_STEPS,
    end_value=1e-6
)
optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adamw(learning_rate=schedule, weight_decay=0.1)
)

# Configs
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=500, max_to_keep=2
)
metrics_logging_options = metrics_logger.MetricsLoggerOptions(
    log_dir="/tmp/grpo_logs", flush_every_n_steps=20
)

# Cluster Configuration
cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: mesh,
        rl_cluster_lib.Role.REFERENCE: mesh,
        rl_cluster_lib.Role.ROLLOUT: mesh,
    },
    rollout_engine='vanilla',
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=10,
        max_steps=GRPO_STEPS,
        mini_batch_size=TRAIN_MICRO_BATCH_SIZE,
        train_micro_batch_size=TRAIN_MICRO_BATCH_SIZE,
        metrics_logging_options=metrics_logging_options,
        checkpoint_root_directory=GRPO_OUTPUT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
         max_tokens_to_generate=400,
         max_prompt_length=256,
         kv_cache_size=1024,
         temperature=0.9, top_p=1.0, top_k=50
    ),
)

grpo_config = GRPOConfig(
    num_generations=4,
    num_iterations=1,
    beta=0.08,
    epsilon=0.2,
)

# Create Cluster
rl_cluster = rl_cluster_lib.RLCluster(
    actor=lora_policy,
    reference=ref_model,
    tokenizer=tokenizer,
    cluster_config=cluster_config,
)


# Trainer
trainer = GRPOLearner(
    rl_cluster=rl_cluster,
    reward_fns=[soft_structure_reward, structure_reward, math_correctness_reward, code_correctness_reward],
    algo_config=grpo_config,  # v0.1.5 API uses 'algo_config'
)

# Data Formatting & Training
with mesh:
    def format_fn(x):
        return {
            "prompts": TEMPLATE.format(question=x["question"]),
            "question": x["question"],
            "answer": x["answer"]
        }
    
    train_ds = grpo_dataset.map(format_fn)
    
    # Custom Batch Generator
    import itertools
    import numpy as np

    def batched(iterable, n):
        it = iter(iterable)
        while True:
            chunk = list(itertools.islice(it, n))
            if not chunk: return
            # Convert list of dicts to dict of numpy arrays
            batch = {k: np.array([d[k] for d in chunk]) for k in chunk[0]}
            yield batch

    def infinite_batch_generator(ds):
        while True:
            # Shuffle each pass for better training
            for batch in batched(ds.shuffle(seed=int(time.time())), TRAIN_MICRO_BATCH_SIZE):
                yield batch

    # Start Training
    trainer.train(infinite_batch_generator(train_ds))

print("GRPO Completed.")


In [ ]:

# --- Visual Sanity Check ---
print("Running Inference on 2 examples...")

# Create Sampler using the trained policy (in memory)
try:
    inference_sampler = sampler_lib.Sampler(
        transformer=lora_policy,
        tokenizer=tokenizer,
        cache_config=sampler_lib.CacheConfig(
            cache_size=MAX_GENERATION_STEPS + 512, # Buffer
            num_layers=model_config.num_layers,
            num_kv_heads=model_config.num_kv_heads,
            head_dim=model_config.head_dim,
        ),
    )

    prompts = [
        "Janet has 3 apples. She buys 2 more. How many does she have?",
        "Write a python function to add two numbers."
    ]
    
    formatted_prompts = [TEMPLATE.format(question=p) for p in prompts]
    
    out_data = inference_sampler(
        input_strings=formatted_prompts,
        max_generation_steps=200,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        echo=False
    )
    
    for p, o in zip(prompts, out_data.text):
        print(f"Prompt: {p}\nOutput: {o}\n{'-'*20}")

except Exception as e:
    print(f"Visual check failed: {e}")


## [Optional 15pts] unrestricted mode

In [ ]:

# --- Save Final Model for Unrestricted Mode ---
# To get the 15 bonus points, you must produce a loadable Kaggle model ID.
# Since you can't upload during a run, save the files here.
# Then, in a separate step (manual via Kaggle UI or API), create the Model from the output.

FINAL_SAVE_DIR = "final_submission_model"
os.makedirs(FINAL_SAVE_DIR, exist_ok=True)

# Save the trained LoRA policy checkpoint
checkpointer = ocp.StandardCheckpointer()
checkpointer.save(os.path.join(FINAL_SAVE_DIR, "checkpoint"), nnx.state(lora_policy, nnx.LoRAParam))
checkpointer.wait_until_finished()

print(f"✅ Model saved to '{FINAL_SAVE_DIR}/'. To submit for Unrestricted Mode:")
print("   1. Download the output folder after this notebook finishes.")
print("   2. Go to Kaggle -> Models -> New Model -> Upload the checkpoint files.")
print("   3. Set the Model ID below to match your upload.")

# Example: 'windmaple/gpt2' in https://www.kaggle.com/models/windmaple/gpt2
# If applying for this, set your Model ID here:
unrestricted_kaggle_model = "yuyamukai/tunix-gemma2-2b-zero-cost"  



## Other things you want the judges to know

### 1. Learnings
*   **SFT is Crucial for RL**: We found that jumping straight to GRPO led to unstable formatting. A short "Format Alignment" SFT phase on Magpie data was essential to teach the model *how* to output specific XML tags before optimizing *what* inside them.
*   **Zero-Cost Feasibility**: It is fully possible to fine-tune a reasoning model on a single TPU v5e-8 within 9 hours using Tunix's efficient `GRPOLearner`.

### 2. Challenges
*   **Version Pinning**: We encountered API mismatches between the `google-tunix` PyPI package and the bleeding-edge GitHub repo. We resolved this by explicitly pinning `google-tunix[prod]==0.1.5` to ensure reproducibility.
*   **Silent Failures**: We identified a critical potential failure where LoRA weights could remain uninitialized if `rngs` weren't properly passed to `nnx` modules. We patched this in our script.

### 3. Feature Requests / Improvements
*   **Unified Config**: The transition from `algo_config` to `config` in `GRPOLearner` was confusing. A stricter, more stable API usage guide for Kaggle would be helpful.
*   **SGLang Support**: We are excited about v0.1.4's SGLang integration, which could double our throughput. We plan to use this in future iterations.
